In [1]:
from brightway2 import *

In [3]:
projects.set_current("Test") # with ecoinvent database loaded

In [5]:
db = Database("cutoff38")

In [12]:
process_list = db.search('electricity, high voltage, production mix', limit=10000000)

In [50]:
target_process_list = [p for p in process_list if 'electricity, high voltage, production mix' in str(p)]

In [61]:
CC_method = [m for m in methods if 'IPCC 2013' in str(m) and  'climate change' in str(m) and 'GWP 100a' in str(m) and not 'no LT' in str(m)][0]
results = {}
for i in range(len(target_process_list)):
    functional_unit = {target_process_list[i]: 1}
    lca = LCA(functional_unit, CC_method)
    lca.lci()
    lca.lcia()
    results[dict(target_process_list[i])['location']] = lca.score


In [64]:
results

{'IN-Northern grid': 1.0677367302260958,
 'IN-North-eastern grid': 0.6123313849107491,
 'CN-CSG': 0.6598178935980432,
 'CN-SGCC': 1.0143169619072647,
 'IN-Eastern grid': 1.4842452294661452,
 'IN-Southern grid': 1.167274019203486,
 'IN-Western grid': 1.3390838216057808,
 'SS': 0.9150586447310122,
 'ZA': 1.0536215414368693,
 'TT': 0.5873098018569624,
 'KH': 0.5024268820195505,
 'ER': 0.9107063389097284,
 'NI': 0.4479708389879745,
 'NZ': 0.09913156922388967,
 'TG': 0.7350837294604526,
 'CL': 0.5547628325625296,
 'DZ': 0.5728952020370499,
 'NG': 0.45124534564203433,
 'IR': 0.6039933906988977,
 'GI': 0.9150586447310122,
 'JP': 0.6624501408438142,
 'CM': 0.05175019377801923,
 'PH': 0.6979577602635494,
 'DO': 0.6947082451569656,
 'IS': 0.04314650487080552,
 'MT': 0.3927213477384933,
 'CD': 0.0048528638881824475,
 'JO': 0.49411080146035646,
 'AO': 0.20222551412433049,
 'VE': 0.28452092987705035,
 'RU': 0.6594171375547289,
 'EG': 0.5403285128298794,
 'ET': 0.005092430747102992,
 'NP': 0.0044587